# Fantasy Premier League 2020/2021 
Basert på følgende tutorial:  
https://towardsdatascience.com/fantasy-premier-league-value-analysis-python-tutorial-using-the-fpl-api-8031edfe9910


## Oppsett

Vi importerer nødvendige biblioteker   
* requests for å laste ned datasettet - https://requests.readthedocs.io/en/master/
* pandas for å behandle data - https://pandas.pydata.org/docs/
* numpy for å behandle data - https://numpy.org/doc/stable/


Datasettet fra Fantasy finnes på følgende link.  
https://fantasy.premierleague.com/api/bootstrap-static/   

#### API - andre nyttige metoder
* Kampprogram: https://fantasy.premierleague.com/api/fixtures/
* Et lags historie: https://fantasy.premierleague.com/api/entry/{}/history/
* Et bestemt lags score i år: https://fantasy.premierleague.com/api/entry/{team-id}/
* Score i en bestemt "classic"-liga: https://fantasy.premierleague.com/api/leagues-classic/{league-id}/standings/
* Score i bestemt "H2H"-liga: https://fantasy.premierleague.com/api/leagues-h2h/{league-id}/standings/
* Siste transfers: https://fantasy.premierleague.com/api/entry/{team-id}/transfers-latest/
* My Team https://fantasy.premierleague.com/api/my-team/{team-id}/ *

*For å bruke "my team api" er autentikasjon påkrevd se link  
https://medium.com/@bram.vanherle1/fantasy-premier-league-api-authentication-guide-2f7aeb2382e4)

#### Andre nyttige lenker
* Reddit-tråd om FPL API  
https://www.reddit.com/r/FantasyPL/comments/c64rrx/fpl_api_url_has_been_changed/
* Blogpost om FPL API (her må man bytte ut drf med api)  
https://medium.com/@YourMumSaysWhat/how-to-get-data-from-the-fantasy-premier-league-api-4477d6a334c3
* Historiske data for FPL  
https://github.com/vaastav/Fantasy-Premier-League

In [41]:
# Nødvendige importer og setup for å komme i gang. 

import requests
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
from IPython.display import display

pd.set_option('display.max_columns', 100)


2020-09-10 20:48:53.371212


### Datasettet lastes ned

Dette endrer seg og det vil være nødvendig å laste ned ganske ofte. Her må hver enkelt finne ut hvordan man ønsker å bruke, og kombinere data. 

In [52]:
# Laster ned datasett som json

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

# Hvis data for i dag finnes; åpne ellers last ned.
if os.path.isfile("{}_fpl_datadump.json".format(datetime.today().date())):
    with open("{}_fpl_datadump.json".format(datetime.today().date())) as json_file:
        fpl_json = json.load(json_file)
else:
    fpl_json = requests.get(url).json() 
    fpl_json["download_time"]= str(datetime.now())
    # dump dataset to local host
    with open("{}_fpl_datadump.json".format(datetime.today().date()), "w") as f:
        json.dump(fpl_json, f, indent=4)

keys = fpl_json.keys()

## Entiteter i datasettet

Dette er et ganske stort datasett,men det er to tydelige interessante objekter;
* elements - som i praksis er spillere
* teams - som inneholder informasjon om lagene
* events - inneholder noe data splittet på gameweek


In [5]:
# Printer nøkler for datasett
for key in keys:
    print(key)

events
game_settings
phases
teams
total_players
elements
element_stats
element_types


In [10]:
# Laster inn ønsket data i frames
teams_df = pd.DataFrame(fpl_json["teams"])
elements_df = pd.DataFrame(fpl_json["elements"])
events_df = pd.DataFrame(fpl_json["events"])


### Teams
Vi visualiserer noen rader av data om lagene og viser alle tilgjengelige nøkler

In [22]:
# Vis data
display(teams_df.head(3))
# Vis tilgjengelige kolonner
display(teams_df.columns.tolist())

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1190,1270,1190,1200,1160,1270,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,2,None,False,0,1050,1080,1010,1080,1000,1110,2
2,36,0,None,3,0,Brighton,0,0,0,BHA,2,None,False,0,1070,1070,1100,1130,1060,1060,131


['code',
 'draw',
 'form',
 'id',
 'loss',
 'name',
 'played',
 'points',
 'position',
 'short_name',
 'strength',
 'team_division',
 'unavailable',
 'win',
 'strength_overall_home',
 'strength_overall_away',
 'strength_attack_home',
 'strength_attack_away',
 'strength_defence_home',
 'strength_defence_away',
 'pulse_id']

,second_name,first_name,team,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,selected_by_percent,special,squad_number,status,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,Özil,Mesut,1,NaN,None,37605,0,0,0,0,0,3,3.4,None,0,0.0,1,False,,None,70,37605.jpg,2.9,1.6,False,None,a,3,53,0,0,0,0,0.0,7.6,Özil,1439,1,3,5,20,0,0,0,1,0,0,1,256,223.6,582.9,190.0,99.8,260,113,36,30,178,98,138,74,3.0,,NaN,,NaN,
1,Papastathopoulos,Sokratis,1,0.0,None,39476,0,0,0,0,0,2,0.0,None,0,0.0,2,False,Calf injury - Expected back 19 Sep,2020-09-11T11:00:08.600094Z,50,39476.jpg,3.0,0.4,False,None,i,3,57,0,0,0,0,0.0,11.4,Sokratis,1696,2,0,4,25,0,0,0,6,0,0,5,305,436.2,36.8,110.0,58.5,146,54,302,107,229,66,239,75,NaN,,NaN,,NaN,
2,Luiz Moreira Marinho,David,1,0.0,None,41270,0,0,0,0,0,2,0.0,None,0,0.0,3,False,Neck - Unknown return date,2020-09-10T14:30:11.632386Z,55,41270.jpg,2.8,2.1,False,None,i,3,94,0,0,0,0,0.0,17.1,David Luiz,2809,2,1,8,42,0,0,0,5,2,0,10,494,701.6,106.7,211.0,102.1,50,18,227,62,164,32,133,35,NaN,,2.0,,NaN,
3,Aubameyang,Pierre-Emerick,1,NaN,None,54694,0,0,0,0,0,3,5.0,None,0,0.0,4,False,,None,120,54694.jpg,5.7,46.2,False,None,a,3,205,0,0,0,0,0.0,17.1,Aubameyang,3136,22,5,10,44,0,0,0,3,1,0,37,807,1006.0,479.6,1369.0,285.2,8,4,56,44,9,3,11,7,NaN,,4.0,,1.0,
4,Soares,Cédric,1,NaN,None,58822,0,0,0,0,0,2,3.1,None,0,0.0,5,False,,None,50,58822.jpg,2.9,0.6,False,None,a,3,61,0,0,0,0,0.0,12.2,Cédric,1553,1,1,4,20,0,0,0,1,0,0,3,286,349.0,218.9,118.0,68.7,186,69,163,35,228,65,207,60,NaN,,NaN,,NaN,


['assists',
 'bonus',
 'bps',
 'chance_of_playing_next_round',
 'chance_of_playing_this_round',
 'clean_sheets',
 'code',
 'corners_and_indirect_freekicks_order',
 'corners_and_indirect_freekicks_text',
 'cost_change_event',
 'cost_change_event_fall',
 'cost_change_start',
 'cost_change_start_fall',
 'creativity',
 'creativity_rank',
 'creativity_rank_type',
 'direct_freekicks_order',
 'direct_freekicks_text',
 'dreamteam_count',
 'element_type',
 'ep_next',
 'ep_this',
 'event_points',
 'first_name',
 'form',
 'goals_conceded',
 'goals_scored',
 'ict_index',
 'ict_index_rank',
 'ict_index_rank_type',
 'id',
 'in_dreamteam',
 'influence',
 'influence_rank',
 'influence_rank_type',
 'minutes',
 'news',
 'news_added',
 'now_cost',
 'own_goals',
 'penalties_missed',
 'penalties_order',
 'penalties_saved',
 'penalties_text',
 'photo',
 'points_per_game',
 'red_cards',
 'saves',
 'second_name',
 'selected_by_percent',
 'special',
 'squad_number',
 'status',
 'team',
 'team_code',
 'threat',

['id',
 'name',
 'deadline_time',
 'average_entry_score',
 'finished',
 'data_checked',
 'highest_scoring_entry',
 'deadline_time_epoch',
 'deadline_time_game_offset',
 'highest_score',
 'is_previous',
 'is_current',
 'is_next',
 'chip_plays',
 'most_selected',
 'most_transferred_in',
 'top_element',
 'top_element_info',
 'transfers_made',
 'most_captained',
 'most_vice_captained']

In [20]:
# Vis 5 første datalinjer
display(events_df[["deadline_time", "finished", "data_checked", "most_transferred_in", "most_captained", "is_next"]].head(5))
# Vis tilgjengelige kolonner
display(events_df.columns.tolist())

,deadline_time,finished,data_checked,most_transferred_in,most_captained,is_next
0,2020-09-12T10:00:00Z,False,False,None,None,True
1,2020-09-19T10:00:00Z,False,False,None,None,False
2,2020-09-26T10:00:00Z,False,False,None,None,False
3,2020-10-03T12:30:00Z,False,False,None,None,False
4,2020-10-17T12:30:00Z,False,False,None,None,False


['id',
 'name',
 'deadline_time',
 'average_entry_score',
 'finished',
 'data_checked',
 'highest_scoring_entry',
 'deadline_time_epoch',
 'deadline_time_game_offset',
 'highest_score',
 'is_previous',
 'is_current',
 'is_next',
 'chip_plays',
 'most_selected',
 'most_transferred_in',
 'top_element',
 'top_element_info',
 'transfers_made',
 'most_captained',
 'most_vice_captained']